# Process GitHub Classroom Students

This script will delete, one by one, students with number `XXXX-1` which represent duplicates. Unfortunately, GH Classroom does not provide bulk deletion or API.

It uses a Salenium Virtual browser.


In [ ]:
import re
import time

# Download geckodriver (https://github.com/mozilla/geckodriver/releases) and put it in path
# Salenium webdriver: https://www.selenium.dev/documentation/overview/
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

Setup variables, including GH classroom to process and GH authentication credentials.

In [ ]:
CLASS_ID="59181697-test-course"
CLASS_ID="139094533-artificial-intelligence-2023"

def get_url(n):
    return f"https://classroom.github.com/classrooms/{CLASS_ID}/roster?roster_entries_page={n}"


USERNAME = '' #Github username - do not commit your own!
PASSWORD = '' #Github password - do not commit your own!

if USERNAME == '' or PASSWORD == '':
    USERNAME = input("Enter username:")
    PASSWORD = input("Enter password:")

Open brower and authenticate to GH:

In [ ]:


# We need an actual browser so that the JavaScript is loaded and the links https://.../games/<game_id> are generated
options = Options()
#options.headless = True
browser = webdriver.Firefox(options=options)
# browser = webdriver.Firefox(options=options, executable_path=r'C:\Utility\BrowserDrivers\geckodriver.exe')
current_page = 1

url = get_url(current_page)
print(f"Extracting web HTML at ", url)
browser.get(url)

#authenticate:
username = browser.find_element(By.NAME, 'login')
username.send_keys(USERNAME)

password = browser.find_element(By.NAME, 'password')
password.send_keys(PASSWORD)

form = browser.find_element(By.NAME,'commit')
form.submit()
time.sleep(10) #let the browser update before we continue


Now do the deletion of users with number `XXXXX-1`, one by one:

In [ ]:

continue_on_current_page = True
# We want to delete all duplicate students from the current page before moving to the next one
# We can't count up front, because the list dynamically changes as we remove duplicates
# We loop on the current page until we do a full pass through the list and find no duplicates

while continue_on_current_page: 
    continue_on_current_page = False
    
    # navigate back to the current page (it always resets to page 1 after deleting any student)
    url = get_url(current_page)
    browser.get(url)
    time.sleep(2) #wait to ensure correct page has loaded


    cards = browser.find_elements(By.CSS_SELECTOR,"div.assignment-list-item.d-flex.col-12")

    if not cards: #if we hit a page with no students we are done and should stop.
        break
    
    for card in cards:

        try:
            description = card.find_element(By.CSS_SELECTOR,"h3.assignment-name-link.h4")
        except:
            #print("No description.")
            continue #sometimes we find an element in the html which is not a student at all, in this case just go to the next one.
        
        #f the student is a duplicate, go through the button clicks to 
        if len(description.text)==9 and description.text[-2:]=="-1": 
            print(f"deleting student: {description.text}")
            button = card.find_elements(By.CSS_SELECTOR, "div.Button-withTooltip")[1]
            button.click()
            time.sleep(2)
            popup= card.find_element(By.CSS_SELECTOR,'input.btn.btn-danger.btn-block.js-submit')
            popup.submit()
            time.sleep(2)
            continue_on_current_page = True #we found a duplicate so should stay on this page
            break #the page has reverted to page 1 so we need to break and reload current page and rescrape list
        else:
            pass
            #print("Not a duplicate")

    #if we made it to the end of the list with no duplicates, we should move on to the next page.
    if continue_on_current_page == False: 
        current_page +=1
        continue_on_current_page = True

